In [34]:
%pip install selenium

import nltk
import requests
import os
from dotenv import load_dotenv
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer

sia = SentimentIntensityAnalyzer()
load_dotenv()

Note: you may need to restart the kernel to use updated packages.


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aadi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [36]:
import subprocess 
import pandas as pd
from nltk.sentiment import SentimentIntensityAnalyzer


# Step 1: Run your scraping scripts 
subprocess. run ( ["python", "../WEB_SCRAPED/scra.py"])
subprocess. run (["python", "../WEB_SCRAPED/collect.py"])

# Step 2: Load news data from the generated CSV
df = pd. read_csv("../WEB_SCRAPED/news_data.csv")

# Step 3: Sentiment Analysis
sia = SentimentIntensityAnalyzer ()

def analyze_sentiment (news_list):
    results = []
    for news in news_list:
        sentiment = sia. polarity_scores (news)
        results.append ({
            "news": news,
            "sentiment_score": sentiment ["compound" ]
    })
    return results

# Analyzing sentiment on the headlines
sentiment_result = analyze_sentiment (df ['Headline']. dropna(). tolist())

# Step 4: Display results including the sentiment
for index, item in enumerate (sentiment_result):
    headline = df. iloc [index] ['Headline']
    description = df. iloc[index] ['Description']
    time = df. iloc[index] ['Time']
    category = df. iloc [index] ['Category']
    sentiment_score = item[ 'sentiment_score']


    print(f"## {headline}\n"
    "Description: {description}\n"
    f"Time: {time}\n"
    f"Category: {category}\n"
    f"→> Sentiment Score: {sentiment_score}\n")

13 items found in business

<div data-testid="card-text-wrapper" class="sc-93223220-0 gcYoBW"><div class="sc-87075214-1 dRanZR"><div class="sc-87075214-0 jvGDZA"><h2 data-testid="card-headline" class="sc-87075214-3 eywmDE">China urges US to 'completely cancel' tariffs<!-- --></h2></div></div><p data-testid="card-description" class="sc-530fb3d6-0 gJqLcg">China, which has retaliated with 125% tariffs, called for a "return to the right path of mutual respect". </p><div class="sc-6fba5bd4-0 gwNQGk"><span data-testid="card-metadata-lastupdated" class="sc-6fba5bd4-1 heskfp">9 hrs ago</span><div data-testid="card-metadata-separator" class="sc-6fba5bd4-3 kVHhvu"></div><span data-testid="card-metadata-tag" class="sc-6fba5bd4-2 jOCsOj">World</span></div></div>


Traceback (most recent call last):
  File "/Users/aadi/GEO_PREDICT/Geo_Predict/notebooks/../WEB_SCRAPED/scra.py", line 31, in <module>
    with open(f"./data/{item}_{file}_{section}.html", "w", encoding="utf-8") as f:
FileNotFoundError: [Errno 2] No such file or directory: './data/article_0_business.html'


## Salah laps up contract acclaim - now Van Dijk may follow 'next week'
Description: {description}
Time: 11 hrs ago
Category: Liverpool
→> Sentiment Score: 0.0

## Australia's looming election brings housing crisis into focus
Description: {description}
Time: 2 hrs ago
Category: Australia
→> Sentiment Score: -0.6808

## Severe floods hit Lanzarote after torrential rain
Description: {description}
Time: 12 hrs ago
Category: Europe
→> Sentiment Score: -0.3818

## How industry leaders transformed their career
Description: {description}
Time: 17 Jan 2024
Category: Work Culture
→> Sentiment Score: 0.0

## Relatives charged after boy, nine, killed in accidental shooting
Description: {description}
Time: 1 hr ago
Category: Australia
→> Sentiment Score: -0.765

## 'Pretty diabolical' - What Australians think of the housing crisis this election
Description: {description}
Time: 1 hr ago
Category: Australia
→> Sentiment Score: -0.2263

## Four simple hacks to succeed in every conversation
Descriptio

Traceback (most recent call last):
  File "/Users/aadi/GEO_PREDICT/Geo_Predict/notebooks/../WEB_SCRAPED/collect.py", line 8, in <module>
    for file in os.listdir(folder):
FileNotFoundError: [Errno 2] No such file or directory: './WEB_SCRAPED/data'


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GOOGLE_API_KEY"] = os.getenv("gem_key")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite-preview-02-05", temperature=0.6)

In [ ]:
def get_llm_relevance(news_text):
    prompt=f"Is the following news article likely to influence stock markets?If yes,state yes and explain how.\n\nArticle:{news_text}"
    response=llm.invoke(prompt)
    return response.content.strip()

In [ ]:
def filter(news_list):
    relevent_articles=[]
    for article in news_list:
        relevance_score=get_llm_relevance(article)
        if "yes" in relevance_score.lower():
            relevent_articles.append(article)
    return relevent_articles

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
from statistics import mode


def process_and_save(filename="sentiment.csv"):
    news_list=fetch_news()
    relevant_articles=filter(news_list)

    data=[]

    for news in relevant_articles:
        sentiment=sia.polarity_scores(news)
        relevance_explanation=get_llm_relevance(news)

        data.append({
            "timestamp":datetime.now().isoformat(),
            "article":news,
            "sentiment_score":sentiment["compound"],
            "label":"Positive" if sentiment["compound"]>0.2 else "Negative",
            "explanation":relevance_explanation
        })

    df=pd.DataFrame(data)

    writer_header=not os.path.exists(filename)
    df.to_csv(filename,mode='a',index=False,header=writer_header)

In [ ]:
process_and_save()